## Introduction
This tutorial explores a variant of the PageRank algorithm: personalized PageRank. Moreover, the issue of data sparsity in PageRank algorithms is discussed. Recall that PageRank computes the importance scores of nodes in a graph where node importance is the probability that we'll be at that node after a random walk. An important application is ranking webpages (So each node in the graph represents each webpage): the algorithm gives each webpage a PageRank score which indicates its importance/influence.

But the regular PageRank algorithm (throughout this tutorial, I will be calling the PageRank algorithm that we learned in class "regular PageRank") has an assumption that is too unrealistic. Personalized PageRank takes some more realistic aspects into consideration: As the name suggests, it incorporates personalization and gives result specifically for that individual.

This tutorial will cover the following:
- [Review of Regular PageRank](#Review-of-Regular-PageRank)
- [Important Additional Notes on Regular PageRank](#Important-Additional-Notes-on-Regular-PageRank)
- [Personalized PageRank](#Personalized-PageRank)
- [Example Code and Issue of Sparsity in PageRank](#Example-Code-and-Issue-of-Sparsity-in-PageRank)

## Review of Regular PageRank

Let $X_{ij}$ denote the row $i$ & column $j$ entry of matrix $X$.

Let $n$ be the number of nodes in the graph.

Define the $n \times n$ matrix $P$ as: $\forall i,j \in \{1, \cdots, n \}$
- If node $i$ has no outgoing edge, then row $i$ of $P$ is defined as all the $n$ entries of the row being $\frac{1}{n}$.
- If node $i$ has an outgoing edge(s), then row $i$ of $P$ is defined by 
    - $P_{ij}$ = $0 \quad$ if there is no edge from node $i$ to node $j$
    - $P_{ij}$ = $\frac{1}{\text{number of outgoing edges of the node $i$}} \quad$ otherwise

Intuitively, $P_{ij}$ is the transition probability of moving from webpage $i$ to webpage $j$. 

Then the regular PageRank algorithm is: initializing an $n \times 1$ vector $\vec{x}$ that sums to $1$, and then iteratively updating $\vec{x}$ until convergence using this update formula: $$ \vec{x} \leftarrow \Big( (1-d) P + d \begin{bmatrix} \frac{1}{n} & \cdots & \frac{1}{n} \\ \vdots & \ddots & \vdots \\ \frac{1}{n} & \cdots & \frac{1}{n} \end{bmatrix} \Big) \vec{x} $$

where the pre-specified $d \ne 0$ is the probability of the internet user moving to a random webpage (chooses uniformly among the $n$ webpages) instead of following the transition probabilities in $P$. Here the matrix $P$ is called the "transition matrix", and the other matrix of all $\frac{1}{n}$'s is called the "teleportation matrix".

## Important Additional Notes on Regular PageRank

Here are some important additional notes about regular PageRank that weren't covered in the lecture slides:
- The value $1-d$, which is the probability that the internet user will take the next move according to the transition probabilities, is called the "damping factor". Google's PageRank algorithm uses damping factor of $0.85$ (i.e. uses $d=0.15$).
- We initialize $\vec{x}$ to sum to $1$, each row of $P$ sums to $1$, and each row of the teleportation matrix sums to $1$. So according to our update rule, $\vec{x}$ always sums to $1$.
- Since $\vec{x}$ always sums to $1$, our update rule can be rewritten as: $$ \vec{x} \leftarrow (1-d) P \vec{x} + d \begin{bmatrix} \frac{1}{n} \\ \vdots \\ \frac{1}{n} \end{bmatrix}$$

Lastly, let's discuss why the PageRank vector $\vec{x}$ will converge to a unique vector. The notions of Markov chains are important here.

For a finite state space $S$, a sequence of random variables $\{ X_{t} \}$ on $S$ is a "Markov chain" if the sequence satisfies the Markov assumption: for any timestep $t$, $\forall x_{0}, \cdots, x_{t}, x_{new} \in S$, $P(X_{t+1}=x_{new} | X_{0}=x_{0}, X_{1}=x_{1}, \cdots, X_{t}=x_{t}) = P(X_{t+1}=x_{new} | X_{t}=x_{t}).$ Intuitively, we wander around nodes (which are called "states" in Markov chain context) in a graph according to transition probabilities, and the most recent past is only what matters for the present. [This link](https://brilliant.org/wiki/markov-chains/) contains great explanations about Markov chains.

Here are some important properties that a Markov chain can have:
- A Markov chain is "finite" if the number of states is finite.
- (informally) A Markov chain is "irreducible" if, regardless of the current state, we can reach any state in finite time.
- A state of a Markov chain is "aperiodic" if its period, defined as the greatest common divisor (gcd) of all the possible numbers of steps it takes to start there and return there, is $1$. (Note: If a state has a self-loop, it's possible to start there and return there in $1$ step, so its period is $1$ since the gcd of numbers including $1$ is $1$.) A Markov chain is "aperiodic" if every state is aperiodic.
- A probability distribution across the states of a Markov chain (each entry shows the proportion of time the chain spends in the corresponding state) that doesn't change as the timestep $t$ increments is called a "stationary distribution". Formally, a stationary distribution $\pi$ satisfies: $\pi P = \pi$, where $P$ is the transition matrix of the Markov chain.

**Theorem 1** (Proof [here](https://www.cc.gatech.edu/~mihail/MC_basics_Vigoda_Lec6.pdf)): Any finite, aperiodic, and irreducible Markov chain has a unique stationary distribution.

Recall the regular PageRank. There's a finite number of webpages, so the Markov chain is finite. Moreover, as long as $d \ne 0$, as the diagonal entries of the teleportation matrix indicate, every webpage has a self loop, so the Markov chain is aperiodic, and moreover, as the entire teleportation matrix indicates, every pair of webpages is connected, so the Markov chain is irreducible. Therefore, by **Theorem 1**, this Markov chain has a unique stationary distribution, and thus the regular PageRank algorithm reaches a unique convergence, as desired.

## Personalized PageRank

It's good that regular PageRank converges to a unique vector: the unique stationary distribution $\pi$ contains the PageRank score for each webpage. But in regular PageRank, the assumption about the internet users is too unrealistic. The assumption is that all the internet users in the world follow the same probabilistic rule when deciding which webpage to go to. In real applications, PageRank on webpages is used for identifying important webpages for actual internet users, and because there are millions of users in the world with diverse preferences or interests, it definitely makes more sense to include personalized information into our alogorithm. That is: we want to rank the webpages by taking each individual's preference into consideration.

So we should revise our update rule in regular PageRank by somehow incorporating personalization, and so we need to choose which matrix (the transition matrix or the teleportation matrix) to revise.

The transition matrix $P$ contains information about the connection/relationships among the webpages, which are essential. When a user is at a webpage, how likely he/she will move to some other webpage is influenced by not just the user's characteristic but the connection between two webpages as well. So we decide to leave the transition matrix unchanged and rather revise the teleportation matrix, which is just an $n \times n$ matrix of all $\frac{1}{n}$'s in regular PageRank, to incorporate personalization. Then we would run PageRank separately for each user, and obviously users with different personalized probabilistic distributions yield different PageRank vectors.

Here is the idea.
- Similar to regular PageRank: initialize $\vec{x}$ as an length-$n$ vector that sums to $1$ and then update
- But the teleportation matrix is now personalized. That is: each internet user has his/her own teleportation matrix. The personalized teleportation matrix for user $u$ is:
$\begin{bmatrix} p_{1}^{(u)} & p_{2}^{(u)} & \cdots & p_{n}^{(u)} \\ p_{1}^{(u)} & p_{2}^{(u)} & \cdots & p_{n}^{(u)} \\ \vdots & \vdots & \ddots & \vdots \\ p_{1}^{(u)} & p_{2}^{(u)} & \cdots & p_{n}^{(u)} \end{bmatrix}$, where $\begin{bmatrix} p_{1}^{(u)} & p_{2}^{(u)} & \cdots & p_{n}^{(u)} \end{bmatrix}$, denoted by $p^{(u)}$, is the (pre-specified) probabilistic distribution of visiting each of the $n$ webpages for that particular user (incorporates the user's interest and preference across the webpages) and is called "personalization vector".
- So the update formula in the personalized PageRank for user $u$ is:
$$ \vec{x} \leftarrow \Big( (1-d) P + d \begin{bmatrix} p_{1}^{(u)} & p_{2}^{(u)} & \cdots & p_{n}^{(u)} \\ p_{1}^{(u)} & p_{2}^{(u)} & \cdots & p_{n}^{(u)} \\ \vdots & \vdots & \ddots & \vdots \\ p_{1}^{(u)} & p_{2}^{(u)} & \cdots & p_{n}^{(u)} \end{bmatrix} \Big) \vec{x} $$

But will this algorithm converge to a unique vector? That is: does this Markov chain have a unique stationary distribution? Recall that in regular PageRank, what made the Markov chain aperiodic is every digaonal entry in the teleportation matrix being nonzero (so every webpage has a self loop), and what made the chain irreducible is every entry in the teleportation matrix being nonzero (so every pair of nodes is connected). In personalization context, most of the entries in a real-world personalization vector are $0$, considering the huge number of webpages and the fact that a user tends to be interested in only a small subset of those millions of webpages. So the personalized teleporation matrix would contain many $0$'s, and so aperiodicity and irreducibility are not guaranteed. We need a remedy.

One way of ensuring aperiodicity and irreducibility is to add the matrix of all $\frac{1}{n}$'s in our update rule, so that every webpage has a self loop with nonzero probability and also every pair of webpages is connected. That is: change the update rule to:
$$ \vec{x} \leftarrow \Big( a P + b \begin{bmatrix} p_{1}^{(u)} & p_{2}^{(u)} & \cdots & p_{n}^{(u)} \\ p_{1}^{(u)} & p_{2}^{(u)} & \cdots & p_{n}^{(u)} \\ \vdots & \vdots & \ddots & \vdots \\ p_{1}^{(u)} & p_{2}^{(u)} & \cdots & p_{n}^{(u)} \end{bmatrix} + c A \Big) \vec{x}$$

where $A$ is the $n \times n$ matrix of all $\frac{1}{n}$'s, or equivalently (since $\vec{x}$ sums to $1$),

$$ \vec{x} \leftarrow a P \vec{x} + b \begin{bmatrix} p^{(u)} \vec{x} \\ p^{(u)} \vec{x} \\ \vdots \\ p^{(u)} \vec{x} \end{bmatrix} + c \begin{bmatrix} \frac{1}{n} \cdots \frac{1}{n} \end{bmatrix}^{T}$$

where our choice of $a$, $b$, and $c$ must satisfy $a + b + c=1$ so that $\vec{x}$ always sums to $1$, and also $c \ne 0$ so that aperiodicity and irreducibility are guaranteed. In this form, we don't actually need to store the $n \times n$ personalized teleportation matrix, but just store the $1 \times n$ personalization vector. This is also an action of dealing with sparsity issue because it uses much less memory compared to storing the $n \times n$ matrix.

Now the personalized PageRank algorithm incorporates personalization and also converges to a unique PageRank vector!

## Example Code and Issue of Sparsity in PageRank

Let's examine some Python code for personalized PageRank on synthetically generated data (for the transition matrix and the personalization vectors). Another key point in PageRank algorithms (not just personalized PageRank) is storing the matrices as sparse matrices. There are over 1.5 billion websites on the World Wide Web ([source](http://www.internetlivestats.com/total-number-of-websites/)), and on average, a webpage has 10 out-links ([source](http://infolab.stanford.edu/~ullman/mmds/ch5.pdf)). You can imagine how sparse the transition matrix would be. Also, a user definitely wouldn't be interested in all those websites, so a personalization vector would also be extremely sparse. So storing those huge matrices/vectors (since $n$ is extremely large as just mentioned) as dense matrices (e.g. Numpy ndarrays) would cause a memory error. Instead, we should store them as sparse matrices offered by the Scipy package ([link](https://docs.scipy.org/doc/scipy/reference/sparse.html)) which only store the nonzero values and handle memory much more efficiently. This issue is addressed in the code below.

In [2]:
# Loading libraries
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import norm

In [ ]:
# In our example, we set n = 80000 which is way smaller than the true number of webpages 
# in the world but is big enough to address the issue of sparsity. 
n = 80000

# Moreover, we consider four different users, 
# each with different interests, and thus different personalization vectors. Imagine a (very) simple scenario. 
# Our setting: 
# Assume that user 1 is only interested in (so has nonzero probability in the personalization vector) the first 20000 webpages, 
# user 2 only in the next 20000 pages, user 3 only in the next 20000, and user 4 only in the last 20000. 
# Moreover, each webpage has out-links to 10 other randomly chosen pages, each with transition probability 0.1
# so that the row sums to 1. 

# The following code chunks contain the code and detailed comments on running the personalized PageRank algorithm 
# for each of the four users.

In [3]:
# ISSUE OF DATA SPARSITY! Here, the transition matrix has dimension 80000 x 80000, so it has total 6400000000 entries. 
# When I try to create a Numpy 2d array of size 80000 x 80000 via the code in this cell, 
# I immediately get a memory error. (Yes, even initializing cannot be done)
# (My laptop is Windwos 64-bit and has RAM 8GB)
transition_matrix = np.zeros((n,n))

MemoryError: 

In [3]:
# Instead, we store matrix as a sparse.lil_matrix offered by the Scipy library which efficiently uses memory by storing only the
# nonzero entries and also is more efficient when changing the matrix entries compared to other Scipy sparse matrix formats 
# like sparse.csr_matrix. 

# (Note: You might be wondering why not use csr_matrix and initialize by specifying the rows & columns & values for the
# nonzero entries. That approach is also problematic because specifying the rows and columns each take length 80000 Numpy arrays
# and cause memory error even before calling the sparse.csr_matrix() function to initialize. So although our approach here 
# is relatively slow as we loop through for filling up the nonzero entries, we use this approach since we can avoid memory error)

# Initialize the 80000 x 80000 sparse transition matrix as all 0's
transition_matrix = sparse.lil_matrix((n,n))

# Fill up nonzero entries by randomly selecting ten outlink pages for each webpage and assigning probability 0.1 
for i in range(0, n):
    outlink_pages = np.random.choice(range(0,n), 10, replace=False)
    # If the randomly chosen pages include itself, replace with some other page that wasn't selected
    if i in outlink_pages:
        done_replacing = False
        # Increment until we find a page that wasn't selected 
        # (When we reach 80000, we start from 0. The mod function takes care of this)
        j = i
        while (not done_replacing):
            j = (j+1) % n
            if (j not in outlink_pages):
                outlink_pages[np.where(outlink_pages==i)] = j
                done_replacing = True
    transition_matrix[i, outlink_pages] = 0.1
    
# We convert the transition matrix (currently sparse.lil_Matrix) into a sparse.csr_matrix
# which is more efficient in arithmetic operations compared to other Scipy's sparse matrix formats
transition_matrix = transition_matrix.tocsr()

In [5]:
# Now we create the personalization vector (length 80000) for each of the four users.
# As mentinoned before, we assume: user 1 is only interested in the first 20000 webpages,
# user 2 only in the next 20000 pages, user 3 only in the next 20000, and user 4 only in the last 20000.

# Again, to avoid memory error, we initialize the 1 x 80000 sparse lil matrix of all 0's
# and then fill in the nonzero entries

# Initialize the 1 x 80000 sparse personalization vector for each user
user1_person_vec = sparse.lil_matrix((1,n))
user2_person_vec = sparse.lil_matrix((1,n))
user3_person_vec = sparse.lil_matrix((1,n))
user4_person_vec = sparse.lil_matrix((1,n))

# Fill in the nonzero entries of the personalization vector for each user
for user in [1,2,3,4]:
    # Create 20000 random numbers, each between 0 and 1
    rand = np.random.rand(1, 20000)
    # Divide each of the 20000 numbers by their sum so that they sum to 1
    rand = np.divide(rand, np.sum(rand))
    if user == 1:
        user1_person_vec[0,range(0,20000)] = rand
    elif user == 2:
        user2_person_vec[0,range(20000,40000)] = rand
    elif user == 3:
        user3_person_vec[0,range(40000,60000)] = rand
    else:
        # if user == 4
        user4_person_vec[0,range(60000,80000)] = rand
        
# Note after creating each personalized teleportation matrix, we convert it (currently in sparse.lil_matrix)
# in to sparse.csr_matrix which is more efficient for arithmetic operations compared to other scipy's sparse matrix formats
user1_person_vec = user1_person_vec.tocsr()
user2_person_vec = user2_person_vec.tocsr()
user3_person_vec = user3_person_vec.tocsr()
user4_person_vec = user4_person_vec.tocsr()

In [18]:
######## Personalized PageRank Algorithm for User 1 ##########

# Decide values of a,b,c. For this tutorial, let's use a = 0.8, b = 0.15, c = 0.05.
a = 0.8
b = 0.15
c = 0.05

# Initialize PageRank vector: 80000 x 1 vector of all 1/80000 's
# We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
# is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
user1_pagerank_vec = sparse.csr_matrix(np.ones((n,1)) / n)

# Update the PageRank vector until convergence! Our convergence criterion is to see whether
# the magnitude of the difference of the PageRank vector after an update is 0
converged = False
while (not converged):
    # (deep) copy the current PageRank vector to compare with the updated vector
    old_user1_pagerank_vec = user1_pagerank_vec.copy()
    
    # Update!
    user1_pagerank_vec = a*transition_matrix.dot(user1_pagerank_vec) \
                        + b*sparse.csr_matrix(user1_person_vec.dot(user1_pagerank_vec)[0,0]*np.ones((n,1))) \
                        + c*sparse.csr_matrix((1/n)*np.ones((n,1)))
            
    # Compare with the PageRank vector before the update. Check the magnitude of the difference of the vector
    print("magnitude of difference vector:", norm(user1_pagerank_vec-old_user1_pagerank_vec))
    
    if norm(user1_pagerank_vec-old_user1_pagerank_vec) == 0:
        converged = True

magnitude of difference vector: 4.31238773442e-18
magnitude of difference vector: 4.31238773442e-18
magnitude of difference vector: 3.35407934899e-18
magnitude of difference vector: 3.35407934899e-18
magnitude of difference vector: 2.87492515628e-18
magnitude of difference vector: 2.87492515628e-18
magnitude of difference vector: 1.91661677085e-18
magnitude of difference vector: 2.87492515628e-18
magnitude of difference vector: 1.91661677085e-18
magnitude of difference vector: 2.39577096357e-18
magnitude of difference vector: 1.91661677085e-18
magnitude of difference vector: 1.91661677085e-18
magnitude of difference vector: 2.39577096357e-18
magnitude of difference vector: 2.39577096357e-18
magnitude of difference vector: 2.39577096357e-18
magnitude of difference vector: 1.91661677085e-18
magnitude of difference vector: 1.91661677085e-18
magnitude of difference vector: 9.58308385427e-19
magnitude of difference vector: 1.91661677085e-18
magnitude of difference vector: 1.91661677085e-18


In [19]:
######## Personalized PageRank Algorithm for User 2 ##########

# Initialize PageRank vector: 80000 x 1 vector of all 1/80000 's
# We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
# is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
user2_pagerank_vec = sparse.csr_matrix(np.ones((n,1)) / n)

# Update the PageRank vector until convergence! Our convergence criterion is to see whether
# the magnitude of the difference of the PageRank vector after an update is 0
converged = False
while (not converged):
    # (deep) copy the current PageRank vector to compare with the updated vector
    old_user2_pagerank_vec = user2_pagerank_vec.copy()
    
    # Update!
    user2_pagerank_vec = a*transition_matrix.dot(user2_pagerank_vec) \
                        + b*sparse.csr_matrix(user2_person_vec.dot(user2_pagerank_vec)[0,0]*np.ones((n,1))) \
                        + c*sparse.csr_matrix((1/n)*np.ones((n,1)))
            
    # Compare with the PageRank vector before the update. Check the magnitude of the difference of the vector
    print("magnitude of difference vector:", norm(user2_pagerank_vec-old_user2_pagerank_vec))
    
    if norm(user2_pagerank_vec-old_user2_pagerank_vec) == 0:
        converged = True

magnitude of difference vector: 2.87492515628e-18
magnitude of difference vector: 3.35407934899e-18
magnitude of difference vector: 2.87492515628e-18
magnitude of difference vector: 1.43746257814e-18
magnitude of difference vector: 1.91661677085e-18
magnitude of difference vector: 2.87492515628e-18
magnitude of difference vector: 1.43746257814e-18
magnitude of difference vector: 1.43746257814e-18
magnitude of difference vector: 1.43746257814e-18
magnitude of difference vector: 4.79154192714e-19
magnitude of difference vector: 9.58308385427e-19
magnitude of difference vector: 9.58308385427e-19
magnitude of difference vector: 1.91661677085e-18
magnitude of difference vector: 9.58308385427e-19
magnitude of difference vector: 9.58308385427e-19
magnitude of difference vector: 1.43746257814e-18
magnitude of difference vector: 1.43746257814e-18
magnitude of difference vector: 9.58308385427e-19
magnitude of difference vector: 9.58308385427e-19
magnitude of difference vector: 4.79154192714e-19


In [20]:
######## Personalized PageRank Algorithm for User 3 ##########

# Initialize PageRank vector: 80000 x 1 vector of all 1/80000 's
# We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
# is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
user3_pagerank_vec = sparse.csr_matrix(np.ones((n,1)) / n)

# Update the PageRank vector until convergence! Our convergence criterion is to see whether
# the magnitude of the difference of the PageRank vector after an update is 0
converged = False
while (not converged):
    # (deep) copy the current PageRank vector to compare with the updated vector
    old_user3_pagerank_vec = user3_pagerank_vec.copy()
    
    # Update!
    user3_pagerank_vec = a*transition_matrix.dot(user3_pagerank_vec) \
                        + b*sparse.csr_matrix(user3_person_vec.dot(user3_pagerank_vec)[0,0]*np.ones((n,1))) \
                        + c*sparse.csr_matrix((1/n)*np.ones((n,1)))
            
    # Compare with the PageRank vector before the update. Check the magnitude of the difference of the vector
    print("magnitude of difference vector:", norm(user3_pagerank_vec-old_user3_pagerank_vec))
    
    if norm(user3_pagerank_vec-old_user3_pagerank_vec) == 0:
        converged = True

magnitude of difference vector: 4.79154192714e-18
magnitude of difference vector: 4.79154192714e-18
magnitude of difference vector: 4.31238773442e-18
magnitude of difference vector: 4.31238773442e-18
magnitude of difference vector: 3.35407934899e-18
magnitude of difference vector: 2.87492515628e-18
magnitude of difference vector: 3.83323354171e-18
magnitude of difference vector: 3.83323354171e-18
magnitude of difference vector: 2.39577096357e-18
magnitude of difference vector: 2.87492515628e-18
magnitude of difference vector: 3.35407934899e-18
magnitude of difference vector: 3.35407934899e-18
magnitude of difference vector: 3.35407934899e-18
magnitude of difference vector: 3.83323354171e-18
magnitude of difference vector: 2.39577096357e-18
magnitude of difference vector: 2.39577096357e-18
magnitude of difference vector: 2.39577096357e-18
magnitude of difference vector: 2.39577096357e-18
magnitude of difference vector: 1.43746257814e-18
magnitude of difference vector: 1.91661677085e-18


In [21]:
######## Personalized PageRank Algorithm for User 4 ##########

# Initialize PageRank vector: 80000 x 1 vector of all 1/80000 's
# We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
# is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
user4_pagerank_vec = sparse.csr_matrix(np.ones((n,1)) / n)

# Update the PageRank vector until convergence! Our convergence criterion is to see whether
# the magnitude of the difference of the PageRank vector after an update is 0
converged = False
while (not converged):
    # (deep) copy the current PageRank vector to compare with the updated vector
    old_user4_pagerank_vec = user4_pagerank_vec.copy()
    
    # Update!
    user4_pagerank_vec = a*transition_matrix.dot(user4_pagerank_vec) \
                        + b*sparse.csr_matrix(user4_person_vec.dot(user4_pagerank_vec)[0,0]*np.ones((n,1))) \
                        + c*sparse.csr_matrix((1/n)*np.ones((n,1)))
            
    # Compare with the PageRank vector before the update. Check the magnitude of the difference of the vector
    print("magnitude of difference vector:", norm(user4_pagerank_vec-old_user4_pagerank_vec))
    
    if norm(user4_pagerank_vec-old_user4_pagerank_vec) == 0:
        converged = True

magnitude of difference vector: 9.58308385427e-19
magnitude of difference vector: 4.79154192714e-19
magnitude of difference vector: 0.0


To verify we have different converged PageRank vectors for the four users, we print, for each pair of the four vectors, the magnitude of the difference. Notice they are all nonzero, so indeed they're all different.

In [22]:
print("Magnitude of difference between PageRank vectors of users 1 and 2:", norm(user1_pagerank_vec-user2_pagerank_vec))
print("Magnitude of difference between PageRank vectors of users 1 and 3:", norm(user1_pagerank_vec-user3_pagerank_vec))
print("Magnitude of difference between PageRank vectors of users 1 and 4:", norm(user1_pagerank_vec-user4_pagerank_vec))
print("Magnitude of difference between PageRank vectors of users 2 and 3:", norm(user2_pagerank_vec-user3_pagerank_vec))
print("Magnitude of difference between PageRank vectors of users 2 and 4:", norm(user2_pagerank_vec-user4_pagerank_vec))
print("Magnitude of difference between PageRank vectors of users 3 and 4:", norm(user3_pagerank_vec-user4_pagerank_vec))

Magnitude of difference between PageRank vectors of users 1 and 2: 8.8643525652e-17
Magnitude of difference between PageRank vectors of users 1 and 3: 1.91661677085e-17
Magnitude of difference between PageRank vectors of users 1 and 4: 5.31861153912e-17
Magnitude of difference between PageRank vectors of users 2 and 3: 1.07809693361e-16
Magnitude of difference between PageRank vectors of users 2 and 4: 3.54574102608e-17
Magnitude of difference between PageRank vectors of users 3 and 4: 7.23522830997e-17


Finally, we show that if we put more weights on the personalized transportation matrix (i.e. use larger value of $b$), the differences of the converged personalized PageRank vectors for the different users are larger than before because more personalization is incorporated.

In [23]:
######## Personalized PageRank Algorithm with larger value of b (more personalization incorporated) ##########

# Decide values of a,b,c. For this tutorial, let's use a = 0.05, b = 0.9, c = 0.05.
a = 0.05
b = 0.9
c = 0.05


##### Personalized PageRank for User 1 #####
# Initialize PageRank vector: 80000 x 1 vector of all 1/80000 's
# We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
# is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
user1_pagerank_vec = sparse.csr_matrix(np.ones((n,1)) / n)

# Update the PageRank vector until convergence! Our convergence criterion is to see whether
# the magnitude of the difference of the PageRank vector after an update is 0
converged = False
while (not converged):
    # (deep) copy the current PageRank vector to compare with the updated vector
    old_user1_pagerank_vec = user1_pagerank_vec.copy()
    
    # Update!
    user1_pagerank_vec = a*transition_matrix.dot(user1_pagerank_vec) \
                        + b*sparse.csr_matrix(user1_person_vec.dot(user1_pagerank_vec)[0,0]*np.ones((n,1))) \
                        + c*sparse.csr_matrix((1/n)*np.ones((n,1)))
            
    if norm(user1_pagerank_vec-old_user1_pagerank_vec) == 0:
        converged = True

##### Personalized PageRank for User 2 #####
# Initialize PageRank vector: 80000 x 1 vector of all 1/80000 's
# We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
# is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
user2_pagerank_vec = sparse.csr_matrix(np.ones((n,1)) / n)

# Update the PageRank vector until convergence! Our convergence criterion is to see whether
# the magnitude of the difference of the PageRank vector after an update is 0
converged = False
while (not converged):
    # (deep) copy the current PageRank vector to compare with the updated vector
    old_user2_pagerank_vec = user2_pagerank_vec.copy()
    
    # Update!
    user2_pagerank_vec = a*transition_matrix.dot(user2_pagerank_vec) \
                        + b*sparse.csr_matrix(user2_person_vec.dot(user2_pagerank_vec)[0,0]*np.ones((n,1))) \
                        + c*sparse.csr_matrix((1/n)*np.ones((n,1)))
            
    if norm(user2_pagerank_vec-old_user2_pagerank_vec) == 0:
        converged = True

##### Personalized PageRank for User 3 #####
# Initialize PageRank vector: 80000 x 1 vector of all 1/80000 's
# We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
# is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
user3_pagerank_vec = sparse.csr_matrix(np.ones((n,1)) / n)

# Update the PageRank vector until convergence! Our convergence criterion is to see whether
# the magnitude of the difference of the PageRank vector after an update is 0
converged = False
while (not converged):
    # (deep) copy the current PageRank vector to compare with the updated vector
    old_user3_pagerank_vec = user3_pagerank_vec.copy()
    
    # Update!
    user3_pagerank_vec = a*transition_matrix.dot(user3_pagerank_vec) \
                        + b*sparse.csr_matrix(user3_person_vec.dot(user3_pagerank_vec)[0,0]*np.ones((n,1))) \
                        + c*sparse.csr_matrix((1/n)*np.ones((n,1)))
            
    if norm(user3_pagerank_vec-old_user3_pagerank_vec) == 0:
        converged = True
        
##### Personalized PageRank for User 4 #####
# Initialize PageRank vector: 80000 x 1 vector of all 1/80000 's
# We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
# is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
user4_pagerank_vec = sparse.csr_matrix(np.ones((n,1)) / n)

# Update the PageRank vector until convergence! Our convergence criterion is to see whether
# the magnitude of the difference of the PageRank vector after an update is 0
converged = False
while (not converged):
    # (deep) copy the current PageRank vector to compare with the updated vector
    old_user4_pagerank_vec = user4_pagerank_vec.copy()
    
    # Update!
    user4_pagerank_vec = a*transition_matrix.dot(user4_pagerank_vec) \
                        + b*sparse.csr_matrix(user4_person_vec.dot(user4_pagerank_vec)[0,0]*np.ones((n,1))) \
                        + c*sparse.csr_matrix((1/n)*np.ones((n,1)))
            
    if norm(user4_pagerank_vec-old_user4_pagerank_vec) == 0:
        converged = True

In [24]:
# Differences of the personalized PageRank vectors when we incorporate more personalization
# All of these differences (except for one in the form of xxx-e17) 
# are in the form of xxx-e16, while all the differences when we incorporated less personalization 
# (except for one in the form of xxx-e16) were in the form of xxx-e17.
# So these appear larger than what we saw before, as desired!
print("Magnitude of difference between PageRank vectors of users 1 and 2:", norm(user1_pagerank_vec-user2_pagerank_vec))
print("Magnitude of difference between PageRank vectors of users 1 and 3:", norm(user1_pagerank_vec-user3_pagerank_vec))
print("Magnitude of difference between PageRank vectors of users 1 and 4:", norm(user1_pagerank_vec-user4_pagerank_vec))
print("Magnitude of difference between PageRank vectors of users 2 and 3:", norm(user2_pagerank_vec-user3_pagerank_vec))
print("Magnitude of difference between PageRank vectors of users 2 and 4:", norm(user2_pagerank_vec-user4_pagerank_vec))
print("Magnitude of difference between PageRank vectors of users 3 and 4:", norm(user3_pagerank_vec-user4_pagerank_vec))

Magnitude of difference between PageRank vectors of users 1 and 2: 4.34592852791e-16
Magnitude of difference between PageRank vectors of users 1 and 3: 7.28314372925e-17
Magnitude of difference between PageRank vectors of users 1 and 4: 2.630556518e-16
Magnitude of difference between PageRank vectors of users 2 and 3: 5.07424290084e-16
Magnitude of difference between PageRank vectors of users 2 and 4: 1.71537200991e-16
Magnitude of difference between PageRank vectors of users 3 and 4: 3.35887089092e-16


## References

1. More on Google's PageRank: http://www.it.uu.se/edu/course/homepage/projektTDB/vt04/projekt5/website/report.pdf
2. More on personalized PageRank: https://nlp.stanford.edu/pubs/haveliwala2003personalizing.pdf
3. (Another variant of PageRank not covered in this tutorial) Topic-sensitive PageRank: http://www-cs-students.stanford.edu/~taherh/papers/topic-sensitive-pagerank.pdf
4. More on Markov chains: https://brilliant.org/wiki/markov-chains/